In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils 

import cv2
import glob

from PIL import Image
#from torchvision import transforms, utils
import random

In [2]:
mnist_test_input = np.load("FASHION_MNIST/testImages.npy")

mnist_train_input = np.load("FASHION_MNIST/trainImages.npy")




train_input = torch.Tensor(mnist_train_input.reshape(mnist_train_input.shape[0], 1, 28, 28)/128.0 - 1.0)#transform_to_tensor( transform, mnist_train_input)
test_input  = torch.Tensor(mnist_test_input.reshape(mnist_test_input.shape[0], 1, 28, 28)/128.0 - 1.0)#transform_to_tensor( transform, mnist_test_input)


In [3]:
# motivated by https://github.com/pytorch/examples/blob/master/dcgan/main.py
class Generator(nn.Module): 
    # input_size -> 28 x 28
    def __init__(self, input_size, channels):
        super(Generator, self).__init__()
        
        
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            # 28 -> 24
            
            nn.ConvTranspose2d(     input_size, channels * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(channels * 8),
            nn.ReLU(True),
            
            # 24 -> 11
            # state size. (channels*8) x 4 x 4
            nn.ConvTranspose2d(channels * 8, channels * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 4),
            nn.ReLU(True),
            
            # 
            # state size. (channels*4) x 8 x 8
            nn.ConvTranspose2d(channels * 4, channels * 2, 4, 2, 2, bias=False),
            nn.BatchNorm2d(channels * 2),
            nn.ReLU(True),
            # state size. (channels*2) x 16 x 16
            nn.ConvTranspose2d(channels * 2, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. 1 x 32 x 32
        )
    
    def forward(self, x): 
        
        ret = self.main(x)
        
        
        return ret
    

class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is 1 x 32 x 32
            nn.Conv2d(1, channels * 2, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(channels * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (channels*2) x 16 x 16
            nn.Conv2d(channels * 2, channels * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (channels*4) x 8 x 8
            nn.Conv2d(channels * 4, channels * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (channels*8) x 4 x 4
            nn.Conv2d(channels * 8, 1, 3, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):    
        
        ret = self.main(x)
        
        
        return ret

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")    

chanNum = 32
nz = 100
    
netG = Generator(nz, chanNum).to(device)
netD = Discriminator(chanNum).to(device)
batch_size = 500
epoch = 300

trlr = 0.0001

optimizerD = optim.Adam(netD.parameters(), lr=trlr)
optimizerG = optim.Adam(netG.parameters(), lr=trlr)

criterion = nn.BCELoss()#F.binary_cross_entropy_with_logits

counter = 0


fn = torch.randn(1, nz, 1, 1, device=device)
cpu = torch.device("cpu")


for lll in range(epoch):
    print("epoch :" + str(lll + 1) )
    
    a = train_input.to(cpu).numpy()
    np.random.shuffle(a)
    train_input = torch.Tensor(a).to(device)
    
    for i in range(0, train_input.shape[0], batch_size):
        
    
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        
        data = train_input[i:i + batch_size]
        
        netD.zero_grad()
        real_cpu = data.to(device)
        
        label = torch.full((batch_size,), 1, device=device)
        label = label.view(-1, 1, 1, 1)
        
        output = netD(real_cpu)
        
        #output = output.view(-1)
        
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(1)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        if i%(batch_size*10) == 0:
            print(errD.item(), errG.item())
            
    
    output_sample = netG(fn).to(cpu).detach().numpy().reshape(28, 28)     
    print(np.max(output_sample))
    print(np.min(output_sample))
    cv2.imwrite("output/" + str(counter) + ".jpg",(output_sample + 1) * 128)
    counter = counter + 1
    


epoch :1
1.4316208362579346 0.7358301281929016
0.659071683883667 1.212134599685669
0.4364740252494812 1.6992148160934448
0.43485206365585327 1.8206164836883545
0.30650460720062256 2.5333917140960693
0.25280851125717163 2.5114850997924805
0.16036108136177063 3.2824087142944336
0.09568323194980621 3.8020474910736084
0.08990531414747238 3.7877230644226074
0.08643126487731934 4.143373489379883
0.09570324420928955 3.7999255657196045
0.13976271450519562 4.176907539367676
0.99900866
-0.99938595
epoch :2
0.07493044435977936 4.001561641693115
0.04823342338204384 4.161825180053711
0.05839165300130844 4.105474472045898
0.07973358035087585 4.007650375366211
0.1253695785999298 3.602494955062866
0.12094011902809143 4.096574783325195
0.0974620133638382 4.168840408325195
0.1654464602470398 3.7749087810516357
0.1345086693763733 3.9264166355133057
0.15112170577049255 4.155726432800293
0.15488091111183167 4.1695780754089355
0.13536600768566132 3.9931302070617676
0.99857295
-0.98878455
epoch :3
0.17732959

0.10835011303424835 4.774771213531494
0.0894707441329956 4.61442756652832
0.9921329
-1.0
epoch :18
0.09152190387248993 4.63126802444458
0.08701274544000626 4.569425582885742
0.09600891172885895 5.4669575691223145
0.12433990836143494 5.205055236816406
0.13930538296699524 4.811977863311768
0.09311375021934509 4.52333927154541
0.1202971488237381 5.300627708435059
0.10569653660058975 4.497644901275635
0.082724928855896 4.575112342834473
0.1483995020389557 4.592124938964844
0.20760829746723175 3.7751595973968506
0.20619668066501617 3.385319232940674
0.98729986
-1.0
epoch :19
0.14390096068382263 4.444514274597168
0.12534251809120178 4.37481689453125
0.1342550367116928 3.8914685249328613
0.11400960385799408 4.437553882598877
0.06887462735176086 5.226293087005615
0.07691274583339691 4.324364185333252
0.16068245470523834 4.103302478790283
0.1311262547969818 4.133864879608154
0.14016908407211304 3.9866578578948975
0.16288113594055176 3.6883442401885986
0.0973798930644989 4.342307090759277
0.0718

0.16367106139659882 4.683234214782715
0.986869
-1.0
epoch :35
0.1698710322380066 4.648143768310547
0.16450189054012299 4.995297431945801
0.09760317206382751 4.918529510498047
0.2583012282848358 3.252876043319702
0.22536182403564453 2.9004244804382324
0.28504014015197754 4.952608585357666
0.19066917896270752 3.764265537261963
0.23083072900772095 3.664963960647583
0.5043160319328308 3.4336133003234863
0.22583813965320587 4.5197014808654785
0.16821883618831635 4.766597747802734
0.17544867098331451 3.6493325233459473
0.99184495
-1.0
epoch :36
0.17388194799423218 4.364587783813477
0.194063201546669 4.055471897125244
0.1774035096168518 4.432880878448486
0.1765579879283905 4.197071075439453
0.1731787621974945 4.317951202392578
0.1713915765285492 4.068342208862305
0.15846586227416992 4.0209479331970215
0.22002607583999634 4.646841526031494
0.2363579273223877 4.550103187561035
0.2328253835439682 4.025841236114502
0.24191763997077942 3.4009361267089844
0.1460285633802414 4.3904571533203125
0.980

0.22933292388916016 4.023043632507324
0.14232781529426575 3.7679474353790283
0.13993215560913086 4.8987627029418945
0.25433245301246643 4.1131463050842285
0.20593228936195374 3.353971004486084
0.24204160273075104 4.246944427490234
0.21984577178955078 3.8963656425476074
0.36803966760635376 3.4001669883728027
0.25038397312164307 4.287646770477295
0.20515385270118713 4.047323703765869
0.22781841456890106 4.410597324371338
0.17355892062187195 4.134847640991211
0.9765676
-1.0
epoch :53
0.17953956127166748 4.345069408416748
0.2720740735530853 4.83457612991333
0.24386325478553772 3.082872152328491
0.38682079315185547 3.9523870944976807
0.36586445569992065 3.432088613510132
0.3163146674633026 3.6356465816497803
0.31734853982925415 4.577162742614746
0.25188958644866943 3.494896411895752
0.20749682188034058 4.079306125640869
0.3016420006752014 3.6042122840881348
0.2518552541732788 3.520157814025879
0.30098769068717957 3.7649834156036377
0.9831862
-1.0
epoch :54
0.19998010993003845 4.391887187957

0.2851698100566864 2.9580202102661133
0.41086509823799133 3.94913387298584
0.3919852077960968 2.9878153800964355
0.3607994318008423 3.7445061206817627
0.2688831686973572 3.5521669387817383
0.31823277473449707 3.6053385734558105
0.21052409708499908 3.4215636253356934
0.24973827600479126 3.314744710922241
0.25716280937194824 3.2598490715026855
0.25982466340065 3.379462957382202
0.24549637734889984 4.645585536956787
0.2092195600271225 3.602343797683716
0.97973865
-1.0
epoch :70
0.22791531682014465 3.750941514968872
0.1657884418964386 4.193195819854736
0.21257384121418 4.204944133758545
0.22359894216060638 3.9716084003448486
0.24752217531204224 3.5271823406219482
0.24234914779663086 3.5958826541900635
0.38948196172714233 2.9795594215393066
0.3580375909805298 2.9633567333221436
0.44685685634613037 3.1888368129730225
0.29056161642074585 3.687790870666504
0.25748366117477417 3.5110058784484863
0.307615727186203 3.365081787109375
0.9642769
-1.0
epoch :71
0.297122597694397 3.6885337829589844
0.

...

0.485312283039093 2.3644227981567383
0.35073643922805786 2.5614688396453857
0.3356807231903076 2.799532175064087
0.3410272002220154 2.9803407192230225
0.33226436376571655 2.558328866958618
0.39029815793037415 3.2543413639068604
0.5507832169532776 2.620314121246338
0.3808444142341614 2.6152701377868652
0.3777543902397156 2.457578420639038
0.287415474653244 3.1978559494018555
0.97115564
-1.0
epoch :223
0.23465460538864136 3.7156074047088623
0.5240912437438965 2.778078079223633
0.5861186385154724 2.5959956645965576
0.3698335587978363 2.2246901988983154
0.40895068645477295 2.8950181007385254
0.42403218150138855 2.4176504611968994
0.3713347315788269 2.812974691390991
0.3346281051635742 3.1729307174682617
0.48579370975494385 2.304258108139038
0.36733531951904297 2.4210000038146973
0.3766863942146301 2.420883893966675
0.38917315006256104 2.9642114639282227
0.9800777
-1.0
epoch :224
0.406410813331604 2.4642333984375
0.2735012471675873 3.209296941757202
0.3856841027736664 2.716500759124756
0.36

0.5377558469772339 1.8423447608947754
0.439275324344635 2.2593772411346436
0.43053102493286133 2.229349374771118
0.3998044729232788 3.4566242694854736
0.40289026498794556 2.4731428623199463
0.5710355639457703 2.33294677734375
0.37962356209754944 2.9564385414123535
0.48344913125038147 2.671964168548584
0.4902443587779999 2.2790472507476807
0.3003309965133667 2.7463595867156982
0.96230274
-1.0
epoch :240
0.3074266016483307 3.6771762371063232
0.2964032292366028 2.777509927749634
0.36902570724487305 3.2271568775177
0.3229011595249176 3.0950047969818115
0.4069409966468811 3.125213623046875
0.591508150100708 2.070012092590332
0.4483463168144226 2.3583288192749023
0.43595248460769653 2.685840606689453
0.3619786202907562 2.731966972351074
0.2996496558189392 2.92923641204834
0.4456833600997925 2.921848773956299
0.547548770904541 2.9811513423919678
0.9283851
-1.0
epoch :241
0.3872857689857483 2.993931531906128
0.26791128516197205 3.06644606590271
0.28607743978500366 2.8569231033325195
0.32402926

0.36367785930633545 2.9853098392486572
0.5531680583953857 2.3235223293304443
0.45966625213623047 2.328793525695801
0.28303903341293335 3.4557933807373047
0.27126380801200867 3.2057998180389404
0.4082795977592468 2.720059871673584
0.6039770841598511 2.398944854736328
0.36411362886428833 2.7262017726898193
0.30873265862464905 3.202923059463501
0.3199751675128937 3.16330623626709
0.9445655
-1.0
epoch :257
0.2939606308937073 3.126624822616577
0.4298589527606964 2.7758710384368896
0.5893716216087341 2.8947436809539795
0.5235686302185059 2.6923537254333496
0.3035416901111603 2.925286054611206
0.37448862195014954 3.2104480266571045
0.38430553674697876 3.2568740844726562
0.43965065479278564 2.6186983585357666
0.46845537424087524 3.2560887336730957
0.490892231464386 2.3520724773406982
0.2866961359977722 3.2243831157684326
0.3472708761692047 3.0464794635772705
0.96589786
-1.0
epoch :258
0.3949323296546936 2.5310115814208984
0.43625736236572266 3.2550137042999268
0.6810117959976196 2.450283288955

0.40978267788887024 2.638307571411133
0.3784881830215454 2.481544017791748
0.3660603165626526 2.895008087158203
0.327189564704895 3.4501583576202393
0.3389348089694977 2.413222551345825
0.3925330340862274 2.8806567192077637
0.3744097054004669 3.1982531547546387
0.3735140264034271 2.6312150955200195
0.2826058864593506 3.060335397720337
0.958415
-1.0
epoch :274
0.4181640148162842 2.8018431663513184
0.35762447118759155 3.142146110534668
0.39455050230026245 3.052860975265503
0.34764358401298523 2.747032403945923
0.28344160318374634 2.8963208198547363
0.41178423166275024 3.165025472640991
0.3809524178504944 2.5010931491851807
0.38461068272590637 2.7008297443389893
0.32542651891708374 2.6467294692993164
0.3516564667224884 2.749213933944702
0.32919323444366455 2.8176138401031494
0.3687080144882202 2.3315136432647705
0.9518395
-1.0
epoch :275
0.3830723166465759 2.728847026824951
0.3802880048751831 2.79541015625
0.30962252616882324 3.43129825592041
0.2912919521331787 2.7368483543395996
0.343885

0.43688568472862244 2.81313419342041
0.30473050475120544 3.1144142150878906
0.4036283493041992 3.527860641479492
0.49877458810806274 2.3185486793518066
0.32339999079704285 2.4738314151763916
0.2690950632095337 3.008556365966797
0.26290494203567505 3.295670986175537
0.44115525484085083 2.7136013507843018
0.4031570553779602 2.7450079917907715
0.9586066
-1.0
epoch :291
0.3420604467391968 2.4665467739105225
0.41820427775382996 3.458085060119629
0.2352106273174286 2.924032211303711
0.48995161056518555 3.3655893802642822
0.6121689677238464 2.5999042987823486
0.4371878504753113 2.6843349933624268
0.40030956268310547 2.492482900619507
0.31296664476394653 3.0985584259033203
0.3794121742248535 2.9762022495269775
0.3331785798072815 2.65566349029541
0.3597961962223053 2.549156427383423
0.2039945125579834 3.2124195098876953
0.94311154
-1.0
epoch :292
0.36591601371765137 2.731672763824463
0.421312153339386 2.1567227840423584
0.366449773311615 2.524214506149292
0.29683706164360046 3.1222431659698486


In [ ]:
cpu = torch.device( "cpu")    

noise = torch.randn(1, nz, 1, 1, device=device)


fake = netG(noise)

fake = fake.view(32, 32)

fakenp = fake.to(cpu).detach().numpy()



cv2.imwrite( "output/example.jpg", (fakenp + 1.0) * 128 )

In [21]:
torch.save(netG.state_dict(), "netG_FMNIST_100")

